# 🧠 LLM + 회귀 기반 이력서 점수 예측
이 노트북은 이력서를 입력받아 OpenAI의 GPT 모델로 항목별 점수를 추출하고, 이를 회귀 모델로 예측 점수를 출력합니다.

## 처리 절차 

- 이력서 데이터 추출 수행  
  - 이력서 Table 컬럼 구성하기 
- 추출된 이력서 데이터를 기준으로 특성 및 구분화 작업 수행 
  - 이력서의 각 항목에 대해서 그룹을 만들고, 우수 이력서에 대한 기준점을 바탕으로 내부적인 임의의 점수화 처리 작업 수행 
  - 해당 작업은 추후에는 이력서의 각정보에 대한 점수 기반 점수가 필요하다. 
- 각 특성별로 프롬프트 기반 점수화 
- 점수화된 각 점수의 평균값과 합계값을 이용해서 종합 점수를 생성 / 회귀분석을 위한 1차 방정식 구성
  - 편향 및 가중치는 추후 단계적으로 검토 예정 
- 회귀 모델 생성 
- 입력값 ( 이력서 데이터 )에 대한 모델 검증 
- 순환을 통해 결과 검증 ( 예측치 정의 )
- 결과값 표현 ( Streamlit 를 이용한 검출 결과 POC 구성 )

In [ ]:
%%capture

%pip install openai scikit-learn pandas
%pip install --upgrade python-dotenv

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# ✅ 점수 추출용 프롬프트 및 함수 정의
import json

SYSTEM_PROMPT_SCORE = """
당신은 채용 담당자입니다. 이력서를 보고 아래 항목에 대해 0~5점으로 평가해주세요.

항목:
1. 기술 적합성
2. 프로젝트 복잡도
3. 경력 일관성
4. 커뮤니케이션
5. 성장 가능성

응답 형식(JSON):
{
  "기술 적합성": 4,
  "프로젝트 복잡도": 5,
  "경력 일관성": 4,
  "커뮤니케이션": 3,
  "성장 가능성": 5
}
"""

def extract_scores(resume_text: str, model="gpt-4") -> dict:
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT_SCORE},
            {"role": "user", "content": resume_text}
        ],
        temperature=0.3
    )
    try:
        return json.loads(response["choices"][0]["message"]["content"])
    except json.JSONDecodeError:
        return {}

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

df = pd.DataFrame([
    {"기술 적합성": 4, "프로젝트 복잡도": 5, "경력 일관성": 4, "커뮤니케이션": 3, "성장 가능성": 5, "적합도점수": 90},
    {"기술 적합성": 3, "프로젝트 복잡도": 3, "경력 일관성": 2, "커뮤니케이션": 2, "성장 가능성": 3, "적합도점수": 65},
    {"기술 적합성": 5, "프로젝트 복잡도": 5, "경력 일관성": 5, "커뮤니케이션": 5, "성장 가능성": 5, "적합도점수": 99},
])

X = df.drop("적합도점수", axis=1)
y = df["적합도점수"]

reg_model = LinearRegression()
reg_model.fit(X, y)

In [ ]:
def predict_resume_score(resume_text: str) -> float:
    scores = extract_scores(resume_text)
    if not scores or len(scores) != 5:
        return None
    input_df = pd.DataFrame([scores])
    return reg_model.predict(input_df)[0]

In [ ]:
resume_text = """
- 백엔드 개발자 5년 (Python, Django, FastAPI, PostgreSQL)
- 쇼핑몰 검색 시스템, 대용량 쿠폰 API 설계 및 운영
- AWS EC2, S3, RDS 기반 CI/CD 구성
- 사내 기술 세미나 발표 및 블로그 운영
"""
score = predict_resume_score(resume_text)
print(f"예상 적합도 점수: {score:.2f}")

### 평균/합게 기반으로 입력된 이력서를 모델로 정의하여 검증 

In [ ]:
# 평균/합계 기반 종합 점수 계산 함수
from statistics import mean

def compute_final_score(scores: dict) -> dict:
    values = list(scores.values())
    avg_score = mean(values)
    sum_score = sum(values)
    combined_score = avg_score + sum_score  # 초기 예측식 (1차 방정식 기반)
    return {
        "score_avg": avg_score,
        "score_sum": sum_score,
        "final_score": combined_score
    }

In [ ]:
scores = extract_scores(resume_text)
print("항목별 점수:", scores)

if scores:
    result = compute_final_score(scores)
    print("평균 점수:", result["score_avg"])
    print("합계 점수:", result["score_sum"])
    print("종합 점수 (예측값):", result["final_score"])
else:
    print("점수 추출 실패")